<hr>
<div style="background-color: lightgray; padding: 20px; color: black;">
<div>
<img src="https://th.bing.com/th/id/R.3cd1c8dc996c5616cf6e65e20b6bf586?rik=09aaLyk4hfbBiQ&riu=http%3a%2f%2fcidics.uanl.mx%2fwp-content%2fuploads%2f2016%2f09%2fcimat.png&ehk=%2b0brgMUkA2BND22ixwLZheQrrOoYLO3o5cMRqsBOrlY%3d&risl=&pid=ImgRaw&r=0" style="float: right; margin-right: 30px;" width="200"/> 
<font size="5.5" color="8C3061"><b>Design Pattern Singleton</b></font> <br>
<font size="5.5" color="8C3061"><b>Creational Design Patterns in Modern C++</b></font> 
</div>
<div style="text-align: left">  <br>
Edison David Serrano Cárdenas<br>
MSc en Matemáticas Aplicadas; CIMAT - Sede Guanajuato <br>
</div>

</div>
<hr>

# <font color="8C3061" >**Introduction**</font> 

This pattern is used when you want a class to have only one instance. 

Now what kind of classes will require only one instance? 

You may have a component that manages some underlying resource and that resource could be maybe a print spooler, maybe a database connection or even an application configuration.



<div class="alert alert-block alert-info">
<b>Note:</b> Now, if you create a class that manages the underlying resource, then the class should have only one instance because if there are multiple instances, then each instance will store its own state of the underlying resource. 

And when one instance modifies the resource, the other instance will not know about it. So this way the state of the underlying resource may get corrupted or it may fail to provide the service. So that's why this class should have only one instance. </div>

> Ensure a class only has one instance, and provide a global point of access to it.

So you might feel that this makes the Singleton instance behave like a global variable. And yes, you're right, it does behave like a global variable.

<font color="8C3061" >**Implementation**</font> 


- The class is made responsible for its own instance.
- It intercepts the call for construction and returns a single instance.
- Same instance is returned every time.
- Therefore, direct construction of object is disabled.
- The class creates its own instance which is provided to the clients.


<font color="8C3061" >**OMD Diagram:**</font> 


<center>
<img src="./images/uml_singleton.png" width="450"/> 
</center>